In [7]:
# Import các thư viện cần thiết

import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [8]:
df = pd.read_excel('employ_clean.xlsx')

In [9]:

df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce')
df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')

# Tính số năm làm việc (Work Duration)
df['Year_work'] = (df['ExitDate'].fillna(pd.to_datetime('today')) - df['StartDate']).dt.days / 365.25

# Mã hóa các cột phân loại
label_encoder = LabelEncoder()
df['IsEmployeeLeft'] = label_encoder.fit_transform(df['IsEmployeeLeft'])
df['GenderCode'] = label_encoder.fit_transform(df['GenderCode'])

# One-Hot Encoding cho cột 'Performance Score' và 'JobFunctionDescription'
df = pd.get_dummies(df, columns=['Performance Score','JobFunctionDescription'], drop_first=True)

# Chọn đặc trưng và mục tiêu
X = df[['Age', 'GenderCode', 'Year_work', 'Current Employee Rating'] + [col for col in df.columns if 'Performance Score' in col or 'JobFunctionDescription' in col]]
y = df['IsEmployeeLeft']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Bắt đầu tính thời gian huấn luyện
start_time_dt = time.time()

# Xây dựng mô hình Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Dự đoán và tính toán các chỉ số đánh giá cho mô hình Decision Tree
y_pred_dt = dt_model.predict(X_test)

# Tính thời gian huấn luyện
training_time_dt = time.time() - start_time_dt

C:\Users\luan0\AppData\Local\Temp\ipykernel_11688\1398766709.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')


In [10]:
conf_matrix = confusion_matrix(y_test, y_pred_dt)
print(conf_matrix)
# Giải thích các giá trị trong ma trận nhầm lẫn
TN, FP, FN, TP = conf_matrix.ravel()

[[347  75]
 [ 96 382]]


In [5]:
print(f'Training Time (Decision Tree): {training_time_dt:.2f} seconds')
classification_rep_dt = classification_report(y_test, y_pred_dt)
print(classification_rep_dt)

Training Time (Decision Tree): 0.02 seconds
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       422
           1       0.84      0.80      0.82       478

    accuracy                           0.81       900
   macro avg       0.81      0.81      0.81       900
weighted avg       0.81      0.81      0.81       900



In [6]:
from sklearn.metrics import cohen_kappa_score

# Tính Cohen's Kappa
kappa_value = cohen_kappa_score(y_test, y_pred_dt)

print(f'Cohen\'s Kappa: {kappa_value:.4f}')


Cohen's Kappa: 0.6196
